In [1]:
import logging

logging.basicConfig(level=logging.DEBUG)

# DEBUG
# INFO
# WARNING
# ERROR
# CRITICAL

logger = logging.getLogger(__name__)

In [2]:
from src.models import Base
import urllib3
from typing import Any
from src.database import engine
from sqlalchemy.orm import sessionmaker

import src.models as models

from sqlalchemy import (
    Table,
    Column,
    Integer,
    Numeric,
    String,
    Date,
)
import requests

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [3]:
Session = sessionmaker(bind=engine)
session = Session()

Base.metadata.create_all(engine)

2025-09-03 15:42:26,627 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)


INFO:sqlalchemy.engine.Engine:SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)


2025-09-03 15:42:26,628 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-09-03 15:42:26,631 INFO sqlalchemy.engine.Engine SELECT schema_name()


...

INFO:sqlalchemy.engine.Engine:SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))


2025-09-03 15:42:26,659 INFO sqlalchemy.engine.Engine [generated in 0.00186s] ('BASE TABLE', 'VIEW', '_CATALOG', 'nsi')


INFO:sqlalchemy.engine.Engine:[generated in 0.00186s] ('BASE TABLE', 'VIEW', '_CATALOG', 'nsi')


2025-09-03 15:42:26,662 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [4]:
BASE_URL = "https://nsi.ffoms.ru/nsi-int/api"
SCHEMA = "nsi"

### Выгрузка справочника по коду

---

- `/data` - Наполнение справочника
- `/structure` - Структура справочника (Типы данных и комментарии к столбцам)
- `/versions` - Список версий справочника (Название справочника и дата обновления)

---

1. Посмотреть последнюю версию справочника по коду в `/versions`
   - Если справочник есть в `_CATALOG` сравнить версию
     - Если версия текущего совпадает, то выход (`up to date`)
     - Иначе берем оттуда название справочника
   - Если справочника нет в `_CATALOG` добавляем запись (на 2-ом шаге мы создадим структуру)
2. Получить структуру справочника по коду в `/structure`
   - Спарсить типы данных и комментарии для столбцов
   - Подготовить `Metadata` для будующей таблицы
3. Получить наполнение справочника по коду в `/data`
   - При успешной выгрузке всего наполнения:
     - Убрать ненужные столбцы: `SYS_RECORDID`, `version`, `SYS_HASH`
     - Создать таблицу с помощью подготовленной `Metadata`
     - Загрузить данные в созданную таблицу


In [ ]:
from sqlalchemy.types import TypeEngine
from sqlalchemy import Text
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from src.schemas import RefBookMetadata, ColumnMetadata


def exists_in_db(refbook_id: str) -> bool:
    return session.get(models.RefBookState, refbook_id) is not None


def create_refbook_record(refbook_id: str):
    session.add(models.RefBookState(id=refbook_id))
    session.commit()


def is_up_to_date(refbook_id: str) -> bool:
    refbook = session.get(models.RefBookState, refbook_id)

    if not refbook or refbook.version is None:
        return False

    db_version = refbook.version

    response = requests.get(
        f"{BASE_URL}/versions",
        params={"identifier": refbook_id, "size": 1},
        verify=False,
    )

    response.raise_for_status()

    data = response.json()

    assert data.get("list")

    api_version = data["list"][0]["version"]

    return api_version == db_version


def parse_column_type(
    data_type: str, length: int | None, fract_part_len: int | None
) -> type[TypeEngine] | TypeEngine:
    if data_type in ("INTEGER", "REFERENCE"):
        return Integer
    elif data_type == "NUMERIC":
        return Numeric(length, fract_part_len)
    elif data_type in ("VARCHAR", "INTEGERVARCHAR"):
        return String(length)
    elif data_type == "DATE":
        return Date
    else:
        return Text


def get_refbook_metadata(refbook_id: str) -> RefBookMetadata:
    response = requests.get(
        f"{BASE_URL}/versions",
        params={"identifier": refbook_id, "size": 1},
        verify=False,
    )

    try:
        response.raise_for_status()
    except Exception:
        print(response.json()["message"])

    data = response.json()

    table_name = data["list"][0]["passport"]["type"].split("-")[0]
    table_comment = (
        data["list"][0]["passport"]["name"]
        + "\n\n"
        + data["list"][0]["passport"]["law"]
    ).rstrip()
    version = data["list"][0]["version"]
    update_date = data["list"][0]["createDate"]

    response = requests.get(
        f"{BASE_URL}/structure",
        params={"identifier": refbook_id},
        verify=False,
    )

    response.raise_for_status()

    data = response.json()

    columns_metadata = [
        ColumnMetadata(
            name=column["Name"],
            type_=parse_column_type(
                column["DataType"],
                column.get("MaxLength"),
                column.get("MaxIntPartLength"),
            ),
            nullable=column["EmptyAllowed"],
            comment=column["Description"],
        )
        for column in data["Columns"]
    ]

    for key in data["Keys"]:
        if key["type"] == "PRIMARY":
            for column in columns_metadata:
                if column.name == key["field"]:
                    column.primary_key = True
                    break

    return RefBookMetadata(
        table_name=table_name,
        table_comment=table_comment,
        version=version,
        update_date=update_date,
        columns_metadata=columns_metadata,
    )


# Create a session with connection pooling and retry logic
def create_session(pool_connections=10, pool_maxsize=10, max_retries=3):
    session = requests.Session()

    # Configure retry strategy
    retry_strategy = Retry(
        total=max_retries,
        backoff_factor=1,
        status_forcelist=[408, 429, 500, 502, 503, 504],
    )

    # Mount adapters for both http and https
    adapter = HTTPAdapter(
        max_retries=retry_strategy,
        pool_connections=pool_connections,
        pool_maxsize=pool_maxsize,
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    return session


def get_refbook_data(refbook_id: str, size: int = 200) -> list[dict[Any, Any]]:
    rows = []
    page = 0

    with create_session() as session:
        while True:
            page += 1

            response = session.get(
                f"{BASE_URL}/data",
                params={
                    "identifier": refbook_id,
                    "page": page,
                    "size": size,
                },
                verify=False,
            )

            response.raise_for_status()

            data = response.json()

            pages = data["total"] // size + 1

            rows.extend(data["list"])

            print(f"page: {page}/{pages}")

            if page >= pages:
                break

    rows = [
        {
            item["column"]: item["value"]
            for item in row
            if item["column"] not in ("SYS_RECORDID", "version", "SYS_HASH")
        }
        for row in data["list"]
    ]

    return rows


def load_refbook_to_db(id: str):
    if is_up_to_date(id):
        return

    if not exists_in_db(id):
        create_refbook_record(id)

    refbook_metadata = get_refbook_metadata(id)
    refbook_data = get_refbook_data(id, size=10_000)

    # Создаем таблицу
    columns = [
        Column(**column_metadata.model_dump())
        for column_metadata in refbook_metadata.columns_metadata
    ]

    table = Table(
        refbook_metadata.table_name,
        Base.metadata,
        *columns,
        schema=SCHEMA,
        comment=refbook_metadata.table_comment,
    )

    with engine.begin() as conn:
        table.drop(conn, checkfirst=True)
        table.create(conn)
        conn.execute(table.insert(), refbook_data)

In [16]:
ids = ("F031",)

for id in ids:
    load_refbook_to_db(id)

2025-09-03 16:03:16,742 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-09-03 16:03:16,743 INFO sqlalchemy.engine.Engine SELECT nsi.[_CATALOG].id AS [nsi__CATALOG_id], nsi.[_CATALOG].table_name AS [nsi__CATALOG_table_name], nsi.[_CATALOG].version AS [nsi__CATALOG_version], nsi.[_CATALOG].update_date AS [nsi__CATALOG_update_date], nsi.[_CATALOG].comment AS [nsi__CATALOG_comment] 
FROM nsi.[_CATALOG] 
WHERE nsi.[_CATALOG].id = ?


INFO:sqlalchemy.engine.Engine:SELECT nsi.[_CATALOG].id AS [nsi__CATALOG_id], nsi.[_CATALOG].table_name AS [nsi__CATALOG_table_name], nsi.[_CATALOG].version AS [nsi__CATALOG_version], nsi.[_CATALOG].update_date AS [nsi__CATALOG_update_date], nsi.[_CATALOG].comment AS [nsi__CATALOG_comment] 
FROM nsi.[_CATALOG] 
WHERE nsi.[_CATALOG].id = ?


2025-09-03 16:03:16,744 INFO sqlalchemy.engine.Engine [cached since 1250s ago] ('F031',)


...

INFO:sqlalchemy.engine.Engine:INSERT INTO nsi.[ERMO] ([IDMO], [NAM_MOP], [NAM_MOK], [INN], [KPP], [OGRN], [OID_MO], [OKOPF], [OKFS], [ADDR_J], [ADDR_J_GAR], [OKTMO], [EMAIL], [PHONE], [FAX], [DATEBEG], [DATEEND]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?,  ... 898 characters truncated ... ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)


2025-09-03 16:03:28,848 INFO sqlalchemy.engine.Engine [insertmanyvalues 72/72 (unordered)] ('01404000000000000', 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ВЕРСАЛЬ"', 'ООО "ВЕРСАЛЬ"', '5009092799', '500901001', '1145009002082', '1.2.643.5.1.13.13.12.2.50.22903', '12300', '16', '142005, обл Московская, г Домодедово, мкр. Центральный, ул Пирогова, д. 9 к. 5', '6c34c754-243f-4c36-882e-24efad00a97c', '46000000000', 'info@mrt-dmd.ru', '+79251350303', '', '08.04.2025', '31.12.2025', '01404900000000000', 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "БИОТЕСТ"', 'ООО "БИОТЕСТ"', '2632076375', '263201001', '1052600230792', '1.2.643.5.1.13.13.12.2.26.17999', '12300', '16', '357502, край Ставропольский, г Пятигорск, ул Баксанская, д 28', 'cde73cd3-7808-4661-aae3-632018ca15eb', '07000000000', 'BIOTEST-BG@LIST.RU', '+78793389481', '', '01.01.2025', '31.12.2025', '01405200000000000', 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ КЛИНИКА "ОМИКРОН-ТОМОГРАД"', 'ООО КЛИНИКА "ОМИКРОН-ТОМОГРАД"', '7610132755', '7

INFO:sqlalchemy.engine.Engine:[insertmanyvalues 72/72 (unordered)] ('01404000000000000', 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ВЕРСАЛЬ"', 'ООО "ВЕРСАЛЬ"', '5009092799', '500901001', '1145009002082', '1.2.643.5.1.13.13.12.2.50.22903', '12300', '16', '142005, обл Московская, г Домодедово, мкр. Центральный, ул Пирогова, д. 9 к. 5', '6c34c754-243f-4c36-882e-24efad00a97c', '46000000000', 'info@mrt-dmd.ru', '+79251350303', '', '08.04.2025', '31.12.2025', '01404900000000000', 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "БИОТЕСТ"', 'ООО "БИОТЕСТ"', '2632076375', '263201001', '1052600230792', '1.2.643.5.1.13.13.12.2.26.17999', '12300', '16', '357502, край Ставропольский, г Пятигорск, ул Баксанская, д 28', 'cde73cd3-7808-4661-aae3-632018ca15eb', '07000000000', 'BIOTEST-BG@LIST.RU', '+78793389481', '', '01.01.2025', '31.12.2025', '01405200000000000', 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ КЛИНИКА "ОМИКРОН-ТОМОГРАД"', 'ООО КЛИНИКА "ОМИКРОН-ТОМОГРАД"', '7610132755', '761001001', '120760001577

2025-09-03 16:03:28,875 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [7]:
# import aiohttp
# import asyncio
# from typing import List, Dict, Any

# async def fetch_page(session: aiohttp.ClientSession, url: str, params: Dict[str, Any]) -> Dict[str, Any]:
#     async with session.get(url, params=params, ssl=False) as response:
#         response.raise_for_status()
#         return await response.json()

# async def fetch_all_paginated_data(
#     base_url: str,
#     initial_params: Dict[str, Any],
#     page_size: int = 200,
#     max_concurrent_requests: int = 20
# ) -> List[Dict[str, Any]]:
#     """
#     Fetch all paginated data from an API efficiently.

#     Args:
#         base_url: The API endpoint URL
#         initial_params: Initial query parameters (without pagination params)
#         page_size: Number of items per page (default 200)
#         max_concurrent_requests: Maximum concurrent requests (default 20)
#     """
#     all_data = []
#     page = 1  # or 0 if API is 0-indexed
#     has_more = True

#     connector = aiohttp.TCPConnector(limit=max_concurrent_requests)
#     timeout = aiohttp.ClientTimeout(total=3600)  # 1 hour timeout

#     async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
#         while has_more:
#             # Prepare requests for a batch of pages
#             tasks = []
#             for _ in range(max_concurrent_requests):
#                 params = initial_params.copy()
#                 params.update({
#                     'page': page,
#                     'size': page_size  # or whatever the API uses for page size
#                 })
#                 tasks.append(fetch_page(session, base_url, params))
#                 page += 1

#             # Execute the batch
#             batch_results = await asyncio.gather(*tasks, return_exceptions=True)

#             # Process results
#             for result in batch_results:
#                 if isinstance(result, Exception):
#                     print(f"Error fetching page: {result}")
#                     continue

#                 if not result.get('data') or len(result['data']) == 0:
#                     has_more = False
#                     break

#                 all_data.extend(result['data'])

#         return all_data

# # Example usage
# async def main():
#     base_url = "https://nsi.ffoms.ru/nsi-int/api/data"
#     params={"identifier": "F033"}

#     data = await fetch_all_paginated_data(base_url, params)
#     print(f"Fetched {len(data)} total records")

# # Run the async main function
# asyncio.run(main())